# Interpolation

In [1]:
# notebook imports
from IPython.display import display
import numpy as np
from podpac.coordinates import Coordinates, clinspace
from podpac.data import Array
from podpac.data import (Interpolation, INTERPOLATION_METHODS, INTERPOLATION_DEFAULT, INTERPOLATORS, INTERPOLATION_METHODS_DICT)

## List Interpolator Methods and Interpolators

See `INTERPOLATION_METHODS_DICT` to connect the string `INTERPOLATION_METHODS` to the list of supported Interpolators

In [2]:
INTERPOLATION_METHODS

['nearest_preview',
 'nearest',
 'bilinear',
 'cubic',
 'cubic_spline',
 'lanczos',
 'average',
 'mode',
 'gauss',
 'max',
 'min',
 'med',
 'q1',
 'q3',
 'spline_2',
 'spline_3',
 'spline_4']

In [3]:
INTERPOLATORS

[podpac.core.interpolation.interpolators.NearestNeighbor,
 podpac.core.interpolation.interpolators.NearestPreview,
 podpac.core.interpolation.interpolators.Rasterio,
 podpac.core.interpolation.interpolators.ScipyPoint,
 podpac.core.interpolation.interpolators.ScipyGrid]

## Nearest Neighbor

In [4]:
# data source 
source = np.arange(0, 9)
source.resize((3, 3))
native_coords = Coordinates([[0, 5, 10], clinspace('2018-01-01', '2018-01-18', 3)], 
                             dims=['lat', 'time'])

# simple interpolation definition
node = Array(source=source, coordinates=native_coords, interpolation="nearest")

# show node source
node.source

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [5]:
# coordinates to evaluate node
eval_coords = Coordinates([[.7, 1.2, 9], clinspace('2018-01-01', '2018-01-09', 3)], 
                          dims=['lat', 'time'])

# run evaluation
output = node.eval(eval_coords)

# elements are chosen based on their nearest neighbor
output.data

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


array([[0., 0., 1.],
       [0., 0., 1.],
       [6., 6., 7.]])

In [6]:
# show the interpolation class for this node (generated by definition `node.interpolation = 'nearest'`)
node.interpolation_class

Interpolation
	('default',): nearest, ['NearestNeighbor', 'Rasterio', 'ScipyPoint', 'ScipyGrid'], {}

In [7]:
# show the interpolators that were used to do this interpolation
# this is only available *after* a node has been evaluated
node.interpolators

OrderedDict([(('lat', 'time'), NearestNeighbor (nearest))])

#### Spatial Tolerance

In [8]:
# set a space tolerance on interpolation
interpolation = {
    'method': 'nearest',
    'params': {
        'spatial_tolerance': 1.1
    }
}
node = Array(source=source, coordinates=native_coords, interpolation=interpolation)

output = node.eval(eval_coords)

# the 1st element is outside of the space tolerance
output.data

array([[ 0.,  0.,  1.],
       [nan, nan, nan],
       [ 6.,  6.,  7.]])

#### Time Tolerance

In [9]:
# set a time tolerance on interpolation
interpolation = {
        'method': 'nearest',
        'params': {
            'spatial_tolerance': 1.1,
            'time_tolerance': np.timedelta64(1, 'D')
        }
}
node = Array(source=source, coordinates=native_coords, interpolation=interpolation)

output = node.eval(eval_coords)
output.data

array([[ 0., nan,  1.],
       [nan, nan, nan],
       [ 6., nan,  7.]])

## Rasterio Interpolation

Rasterio provides interpolation support for more advanced interpolation methods.
The following interpolation methods are supported by rasterio via podpac:

```python
['nearest', 
 'bilinear', 
 'cubic', 
 'cubic_spline', 
 'lanczos', 
 'average', 
 'mode', 
 'gauss', 
 'max', 
 'min', 
 'med', 
 'q1', 
 'q3']
```

In [10]:
# data source
source = np.arange(0, 15)
source.resize((3, 5))

native_coords = Coordinates([clinspace(0, 10, 3), clinspace(0, 10, 5)], dims=['lat', 'lon'])
node = Array(source=source, coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([clinspace(1, 11, 3), clinspace(1, 11, 5)], dims=['lat', 'lon'])

# show node data
node.source

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [11]:
# max interpolation from rasterio
node = Array(source=source, coordinates=native_coords, interpolation='max')

output = node.eval(eval_coords)
output.data

array([[ 6.,  7.,  8.,  9.,  9.],
       [11., 12., 13., 14., 14.],
       [11., 12., 13., 14., 14.]])

In [12]:
# min interpolation
node = Array(source=source, coordinates=native_coords, interpolation='min')

output = node.eval(eval_coords)
output.data

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.]])

In [13]:
# bilinear
node = Array(source=source, coordinates=native_coords, interpolation='bilinear')

output = node.eval(eval_coords)
output.data

array([[ 1.4,  2.4,  3.4,  4.4,  5. ],
       [ 6.4,  7.4,  8.4,  9.4, 10. ],
       [10.4, 11.4, 12.4, 13.4, 14. ]])

In [14]:
# cubic_spline
node = Array(source=source, coordinates=native_coords, interpolation='cubic_spline')

output = node.eval(eval_coords)
output.data

array([[ 2.01204892,  2.95976676,  3.95976676,  4.94251609,  5.49712175],
       [ 6.44026614,  7.38798398,  8.38798398,  9.3707333 ,  9.92533897],
       [ 9.85637899, 10.80409683, 11.80409683, 12.78684616, 13.34145182]])

## Specify Interpolator (Advanced)

You can specify a single type of interpolator for a data source using the `interpolators` key of the interpolation dictionary definition.

In [15]:
from podpac.interpolators import NearestNeighbor, Rasterio, ScipyGrid, ScipyPoint

In [16]:
# data source
source = np.arange(0, 25)
source.resize((5, 5))

native_coords = Coordinates([clinspace(0, 10, 5), clinspace(0, 10, 5)], dims=['lat', 'lon'])
node = Array(source=source, coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([clinspace(1, 11, 5), clinspace(1, 11, 5)], dims=['lat', 'lon'])

# show node data
node.source

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [17]:
native_coords['lat'].coordinates

array([ 0. ,  2.5,  5. ,  7.5, 10. ])

In [18]:
eval_coords['lat'].coordinates

array([ 1. ,  3.5,  6. ,  8.5, 11. ])

In [19]:
# use Scipy for nearest neighbor
interpolation = {
    'method': 'nearest',
    'interpolators': [ScipyGrid]
}
node = Array(source=source, coordinates=native_coords, interpolation=interpolation)

output = node.eval(eval_coords)
output.data

array([[ 0.,  1.,  2.,  3., nan],
       [ 5.,  6.,  7.,  8., nan],
       [10., 11., 12., 13., nan],
       [15., 16., 17., 18., nan],
       [nan, nan, nan, nan, nan]])

In [20]:
# use Scipy for cubic_spline
interpolation = {
    'method': 'bilinear',
    'interpolators': [ScipyGrid]
}
node = Array(source=source, coordinates=native_coords, interpolation=interpolation)

output = node.eval(eval_coords)
output.data

array([[ 2.4,  3.4,  4.4,  5.4,  nan],
       [ 7.4,  8.4,  9.4, 10.4,  nan],
       [12.4, 13.4, 14.4, 15.4,  nan],
       [17.4, 18.4, 19.4, 20.4,  nan],
       [ nan,  nan,  nan,  nan,  nan]])

In [21]:
# use Scipy for cubic_spline
interpolation = {
    'method': 'cubic_spline',
    'interpolators': [ScipyGrid]
}
node = Array(source=source, coordinates=native_coords, interpolation=interpolation)

output = node.eval(eval_coords)
output.data

array([[ 2.4,  3.4,  4.4,  5.4,  6. ],
       [ 7.4,  8.4,  9.4, 10.4, 11. ],
       [12.4, 13.4, 14.4, 15.4, 16. ],
       [17.4, 18.4, 19.4, 20.4, 21. ],
       [20.4, 21.4, 22.4, 23.4, 24. ]])

## Multiple Dimensions

> This feature still in development

Specify different interpolation methods for each dimension

In [22]:
# data source
source = np.arange(0, 50)
source.resize((2, 5, 5))

native_coords = Coordinates([[0, 10], clinspace(0, 10, 5), clinspace(0, 10, 5)], dims=['lat', 'lon', 'time'])
node = Array(source=source, coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([[5], clinspace(1, 11, 5), clinspace(1, 11, 5)], dims=['lat', 'lon', 'time'])

# show node data
node.source

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]],

       [[25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49]]])

In [ ]:
# NOTE: This feature is still in development and will be fixed in future releases of popdac (>2.0.0)

# use different interpolators on different dimensions
interpolation = [
    {
        'method': 'bilinear',
        'dims': ['lat', 'lon']
    },
    {
        'method': 'nearest',
        'dims': ['time']
    }
]
node = Array(source=source, coordinates=native_coords, interpolation=interpolation)

output = node.eval(eval_coords)